In [1]:
import os
os.getcwd()

'd:\\HuggingFace'

In [2]:
#os.environ['TRANSFORMERS_CACHE'] = 'd:\\HuggingFace\\cache\\'
os.environ['TRANSFORMERS_CACHE'] = 'd:/HuggingFace/cache/huggingface/transformers/'
os.environ['HF_HOME'] = 'd:/HuggingFace/cache/huggingface/transformers/' 

In [3]:
os.environ['TRANSFORMERS_CACHE']

'd:/HuggingFace/cache/huggingface/transformers/'

In [4]:
from transformers import pipeline

In [5]:
classifier = pipeline("sentiment-analysis")
classifier("I've been waiting for a HuggingFace course my whole life.")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)
Downloading: 100%|██████████| 255M/255M [02:03<00:00, 2.18MB/s]


[{'label': 'POSITIVE', 'score': 0.9598049521446228}]

In [6]:
classifier(["This is a great course.", "This wasn't a very nice season"])

[{'label': 'POSITIVE', 'score': 0.9998630285263062},
 {'label': 'NEGATIVE', 'score': 0.9997233748435974}]

In [8]:
from transformers import TFAutoModelForSequenceClassification

In [9]:
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"

In [10]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [11]:
raw_inputs = ["This is a great course.", "This wasn't a very nice season"]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="tf")
print(inputs)

{'input_ids': <tf.Tensor: shape=(2, 10), dtype=int32, numpy=
array([[ 101, 2023, 2003, 1037, 2307, 2607, 1012,  102,    0,    0],
       [ 101, 2023, 2347, 1005, 1056, 1037, 2200, 3835, 2161,  102]])>, 'attention_mask': <tf.Tensor: shape=(2, 10), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])>}


In [14]:
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(inputs)

Some layers from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english and are newly initialized: ['dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
print(outputs.logits)

tf.Tensor(
[[-4.29666    4.5988746]
 [ 4.5151606 -3.6775267]], shape=(2, 2), dtype=float32)


In [17]:
print(model.config.id2label)

{0: 'NEGATIVE', 1: 'POSITIVE'}


In [18]:
import tensorflow as tf
predictions = tf.math.softmax(outputs.logits, axis=-1)
print(predictions)

tf.Tensor(
[[1.3698058e-04 9.9986303e-01]
 [9.9972337e-01 2.7659291e-04]], shape=(2, 2), dtype=float32)


## Section 2 - Tokenizers

In [5]:
from transformers import AutoTokenizer

In [6]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

Downloading: 100%|██████████| 29.0/29.0 [00:00<00:00, 14.1kB/s]
Downloading: 100%|██████████| 570/570 [00:00<00:00, 285kB/s]
Downloading: 100%|██████████| 208k/208k [00:05<00:00, 40.4kB/s]
Downloading: 100%|██████████| 426k/426k [00:04<00:00, 98.0kB/s]


In [9]:
raw_text="Let's try to tokenize! this input"

In [10]:
final_direct = tokenizer(raw_text)
print(final_direct)

{'input_ids': [101, 2421, 112, 188, 2222, 1106, 22559, 3708, 106, 1142, 7758, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [11]:
tokens = tokenizer.tokenize(raw_text)
print(tokens)

['Let', "'", 's', 'try', 'to', 'token', '##ize', '!', 'this', 'input']


In [13]:
input_ids = tokenizer.convert_tokens_to_ids(tokens)
print(input_ids)

[2421, 112, 188, 2222, 1106, 22559, 3708, 106, 1142, 7758]


In [14]:
final_inputs = tokenizer.prepare_for_model(input_ids)
print(final_inputs)

{'input_ids': [101, 2421, 112, 188, 2222, 1106, 22559, 3708, 106, 1142, 7758, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [16]:
decoded_string = tokenizer.decode(final_inputs['input_ids'])
print(decoded_string)

[CLS] Let's try to tokenize! this input [SEP]


In [17]:
assert(final_direct==final_inputs)

In [18]:
final_direct==final_inputs

True

### Using Multiple Sequences

In [5]:
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

In [6]:
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint)

All model checkpoint layers were used when initializing TFDistilBertForSequenceClassification.

All the layers of TFDistilBertForSequenceClassification were initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


In [7]:
sequence = "This is the sequences bit in the HuggingFace course."

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

[2023, 2003, 1996, 10071, 2978, 1999, 1996, 17662, 12172, 2607, 1012]


In [8]:
input_ids = tf.constant([ids])
print(input_ids)

tf.Tensor([[ 2023  2003  1996 10071  2978  1999  1996 17662 12172  2607  1012]], shape=(1, 11), dtype=int32)


In [9]:
output = model(input_ids)
print(output)

TFSequenceClassifierOutput(loss=None, logits=<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[ 2.1915398, -2.017821 ]], dtype=float32)>, hidden_states=None, attentions=None)


In [10]:
batched_ids = [ids, ids]
input_ids = tf.constant(batched_ids)

In [11]:
input_ids = tf.constant(batched_ids)
print("Input: ",input_ids)

output = model(input_ids)
print("Output: ",output)

Input:  tf.Tensor(
[[ 2023  2003  1996 10071  2978  1999  1996 17662 12172  2607  1012]
 [ 2023  2003  1996 10071  2978  1999  1996 17662 12172  2607  1012]], shape=(2, 11), dtype=int32)
Output:  TFSequenceClassifierOutput(loss=None, logits=<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[ 2.1915393, -2.017821 ],
       [ 2.1915393, -2.017821 ]], dtype=float32)>, hidden_states=None, attentions=None)


In [17]:
ls = [1,2,3,4,5]
ls1 = [200]*4
ls = ls + [200]*4
print(ls)
print(ls1)

[1, 2, 3, 4, 5, 200, 200, 200, 200]
[200, 200, 200, 200]


In [13]:
ls1.extend([200]*3)
print(ls)
print(ls1)

[1, 2, 3, 4, 5]
[200, 200, 200, 200, 200, 200, 200]


In [19]:
del ls, ls1
pad_tmp = 100
ls = [[1,2,3,4,5],[1,2,3,4],[1,2,3]]
#ls1 = [x+[pad_tmp]*2 for x in ls]
ls1 = [x+[pad_tmp]*(6-len(x)) for x in ls]
print(ls1)

[[1, 2, 3, 4, 5, 100], [1, 2, 3, 4, 100, 100], [1, 2, 3, 100, 100, 100]]


In [20]:
tokenizer.pad_token_id

0

In [22]:
str_inp_1 = "This is test string 1"
tokens_1 = tokenizer.tokenize(str_inp_1)
sequence1_ids = [tokenizer.convert_tokens_to_ids(tokens_1)]
print(sequence1_ids)

[[2023, 2003, 3231, 5164, 1015]]


In [23]:
str_inp_2 = "And this is the second input we will give"
tokens_2 = tokenizer.tokenize(str_inp_2)
sequence2_ids = [tokenizer.convert_tokens_to_ids(tokens_2)]
print(sequence2_ids)

[[1998, 2023, 2003, 1996, 2117, 7953, 2057, 2097, 2507]]


In [24]:
batched_ids = sequence1_ids+sequence2_ids
print(batched_ids)

[[2023, 2003, 3231, 5164, 1015], [1998, 2023, 2003, 1996, 2117, 7953, 2057, 2097, 2507]]


In [25]:
padding_id = tokenizer.pad_token_id

max_seq_len = max([len(x) for x in batched_ids])
print(max_seq_len)

9


In [26]:
batched_ids = [x+[padding_id]*(max_seq_len-len(x)) for x in batched_ids]
print(batched_ids)

[[2023, 2003, 3231, 5164, 1015, 0, 0, 0, 0], [1998, 2023, 2003, 1996, 2117, 7953, 2057, 2097, 2507]]


In [27]:
print(model(tf.constant(sequence1_ids)).logits)
print(model(tf.constant(sequence2_ids)).logits)
print(model(tf.constant(batched_ids)).logits)


tf.Tensor([[ 3.3556943 -2.727254 ]], shape=(1, 2), dtype=float32)
tf.Tensor([[0.07108341 0.2511712 ]], shape=(1, 2), dtype=float32)
tf.Tensor(
[[ 1.8196992  -1.6044074 ]
 [ 0.0710834   0.25117114]], shape=(2, 2), dtype=float32)


### In the above results, the sequence with no padding is same, as attention mask will be same, but the one with padding is different. As the attention mask also currently covers the padding in the batched_ids bit

### If we don't modify attention masks too, the model will treat the values differently - to demonstrate let's run without attention mask modification

In [29]:
attention_mask = [[0 if k==padding_id else 1 for k in x] for x in batched_ids]
print(attention_mask)

[[1, 1, 1, 1, 1, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1]]


In [30]:
outputs = model(tf.constant(batched_ids),attention_mask = tf.constant(attention_mask))
print(outputs.logits)

tf.Tensor(
[[ 3.3556952  -2.7272546 ]
 [ 0.0710834   0.25117114]], shape=(2, 2), dtype=float32)


In [31]:
print(model(tf.constant(sequence1_ids)).logits)
print(model(tf.constant(sequence2_ids)).logits)

tf.Tensor([[ 3.3556943 -2.727254 ]], shape=(1, 2), dtype=float32)
tf.Tensor([[0.07108341 0.2511712 ]], shape=(1, 2), dtype=float32)


### Now the values are the same

## Chapter 2 - Putting it all together 

In [33]:
from transformers import AutoTokenizer

In [32]:
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

model_inputs = tokenizer(sequence)
print(model_inputs)

{'input_ids': [101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


### The same tokenizer can also handle multiple sequences 


In [35]:
sequences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]
model_inputs = tokenizer(sequences)
print(model_inputs)

{'input_ids': [[101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 102], [101, 2061, 2031, 1045, 999, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}


In [36]:
# Will pad the sequences up to the maximum sequence length
model_inputs = tokenizer(sequences, padding="longest")
print(model_inputs)

# Will pad the sequences up to the model max length
# (512 for BERT or DistilBERT)
model_inputs = tokenizer(sequences, padding="max_length")
print(model_inputs)

# Will pad the sequences up to the specified max length
model_inputs = tokenizer(sequences, padding="max_length", max_length=8)
print(model_inputs)

{'input_ids': [[101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 102], [101, 2061, 2031, 1045, 999, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]}
{'input_ids': [[101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [37]:
sequences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]

# Will truncate the sequences that are longer than the model max length
# (512 for BERT or DistilBERT)
model_inputs = tokenizer(sequences, truncation=True)
print(model_inputs)

{'input_ids': [[101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 102], [101, 2061, 2031, 1045, 999, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}


In [38]:
# Will truncate the sequences that are longer than the specified max length
model_inputs = tokenizer(sequences, max_length=8, truncation=True)
print(model_inputs)

{'input_ids': [[101, 1045, 1005, 2310, 2042, 3403, 2005, 102], [101, 2061, 2031, 1045, 999, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}


In [39]:
sequences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]

# Returns PyTorch tensors
model_inputs = tokenizer(sequences, padding=True, return_tensors="pt")
print(model_inputs)

# Returns TensorFlow tensors
model_inputs = tokenizer(sequences, padding=True, return_tensors="tf")
print(model_inputs)

# Returns NumPy arrays
model_inputs = tokenizer(sequences, padding=True, return_tensors="np")
print(model_inputs)

{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102],
        [  101,  2061,  2031,  1045,   999,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}
{'input_ids': <tf.Tensor: shape=(2, 16), dtype=int32, numpy=
array([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662,
        12172,  2607,  2026,  2878,  2166,  1012,   102],
       [  101,  2061,  2031,  1045,   999,   102,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0]])>, 'attention_mask': <tf.Tensor: shape=(2, 16), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])>}
{'input_ids': array([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 1766

In [40]:
sequence = "I've been waiting for a HuggingFace course my whole life."

model_inputs = tokenizer(sequence)
print(model_inputs["input_ids"])

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

[101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 102]
[1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012]


In [41]:
print(tokenizer.decode(model_inputs["input_ids"]))
print(tokenizer.decode(ids))

[CLS] i've been waiting for a huggingface course my whole life. [SEP]
i've been waiting for a huggingface course my whole life.


### Summary

In [43]:
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

In [42]:
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint)
sequences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]

tokens = tokenizer(sequences, padding=True, truncation=True, return_tensors="tf")
output = model(**tokens)
print(output.logits)

Some layers from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english and are newly initialized: ['dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tf.Tensor(
[[-1.5606971  1.612282 ]
 [-3.6183178  3.9137497]], shape=(2, 2), dtype=float32)
